In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict

In [2]:
class Generator(nn.Module):
    def __init__(self, in_channels, batch_size, kernel_size = (3, 3), pool_size = (2, 2), padding = 0, conv_stride = 1, pool_stride = 2):
        super(Generator, self).__init__()
        
        self.convblock1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(in_channels, 16, kernel_size, conv_stride, padding)),
            ('activation1', nn.ReLU()),
            ('maxpool1', nn.MaxPool2d(pool_size, pool_stride))
        ]))
        
        self.convblock2 = nn.Sequential(OrderedDict([
            ('conv2', nn.Conv2d(16, 32, kernel_size, conv_stride, padding)),
            ('activation2', nn.ReLU()),
            ('maxpool2', nn.MaxPool2d(pool_size, pool_stride))
        ]))
        
        self.convblock3 = nn.Sequential(OrderedDict([
            ('conv3', nn.Conv2d(32, 64, kernel_size, conv_stride, padding)),
            ('activation3', nn.ReLU()),
            ('maxpool3', nn.MaxPool2d(pool_size, pool_stride))
        ]))
        
        self.upblock = nn.Sequential(OrderedDict([
            ('upsample1', nn.Upsample((124, 124))),
            ('transposed_conv1', nn.ConvTranspose2d(64, 32, kernel_size, conv_stride, padding)),
            ('activation1', nn.ReLU()),
            ('upsample2', nn.Upsample((253, 253))),
            ('transposed_conv2', nn.ConvTranspose2d(32, 16, kernel_size, conv_stride, padding)),
            ('activation2', nn.ReLU()),
            ('upsample3', nn.Upsample((510, 510))),
            ('transposed_conv3', nn.ConvTranspose2d(16, 1, kernel_size, conv_stride, padding)),
            ('activation2', nn.Sigmoid())
        ]))
        
    def forward(self, X):
        
        X = self.convblock1(X)
        X = self.convblock2(X)
        X = self.convblock3(X)
        X = self.upblock(X)
        return X

In [3]:
dummy = torch.rand(32, 1, 512, 512)
gen = Generator(1, 32)
out_gen = gen(dummy)

In [6]:
out_gen

tensor([[[[-0.4112, -0.2879, -0.5297,  ..., -0.5268, -0.0872, -0.2138],
          [-0.9616, -0.6704, -0.4586,  ..., -0.4496,  0.5380,  0.2427],
          [-0.7735,  0.4296,  0.7375,  ...,  0.7418,  1.5446,  0.3336],
          ...,
          [-0.7720,  0.4300,  0.7364,  ...,  0.7387,  1.5422,  0.3348],
          [-0.3371,  0.7406,  1.2855,  ...,  1.2864,  1.6513,  0.5713],
          [ 0.2114,  1.1206,  1.2121,  ...,  1.2135,  1.0289,  0.1165]]],


        [[[-0.4111, -0.2878, -0.5298,  ..., -0.5269, -0.0872, -0.2139],
          [-0.9613, -0.6702, -0.4584,  ..., -0.4500,  0.5376,  0.2425],
          [-0.7734,  0.4293,  0.7368,  ...,  0.7414,  1.5441,  0.3335],
          ...,
          [-0.7720,  0.4308,  0.7375,  ...,  0.7386,  1.5427,  0.3346],
          [-0.3375,  0.7405,  1.2850,  ...,  1.2866,  1.6519,  0.5714],
          [ 0.2111,  1.1202,  1.2114,  ...,  1.2136,  1.0291,  0.1165]]],


        [[[-0.4111, -0.2876, -0.5298,  ..., -0.5267, -0.0873, -0.2139],
          [-0.9614, -0.670

In [4]:
class Discriminator(nn.Module):
    def __init__(self, in_channels, batch_size, kernel_size = (3, 3), pool_size = (2, 2), padding = 0, conv_stride = 1, pool_stride = 2):
        super(Discriminator, self).__init__()
        
        self.convblock1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(in_channels, 16, kernel_size, conv_stride, padding)),
            ('activation1', nn.ReLU()),
            ('maxpool1', nn.MaxPool2d(pool_size, pool_stride))
        ]))
        
        self.convblock2 = nn.Sequential(OrderedDict([
            ('conv2', nn.Conv2d(16, 32, kernel_size, conv_stride, padding)),
            ('activation2', nn.ReLU()),
            ('maxpool2', nn.MaxPool2d(pool_size, pool_stride))
        ]))
        
        self.convblock3 = nn.Sequential(OrderedDict([
            ('conv3', nn.Conv2d(32, 64, kernel_size, conv_stride, padding)),
            ('activation3', nn.ReLU()),
            ('maxpool3', nn.MaxPool2d(pool_size, pool_stride))
        ]))
        
        self.linear = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(64*62*62, 256)),
            ('activation1', nn.ReLU()),
            ('linear2', nn.Linear(256, 64)),
            ('activation2', nn.ReLU()),
            ('linear3', nn.Linear(64, 1)),
            ('activation3', nn.Sigmoid())
            
        ]))
        
    def forward(self, X):
        
        X = self.convblock1(X)
        X = self.convblock2(X)
        X = self.convblock3(X)
        X = X.view(-1, 64*62*62)
        X = self.linear(X)
        return X

In [5]:
D = Discriminator(1, 1)
D(out_gen)

tensor([[0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906],
        [0.4906]], grad_fn=<SigmoidBackward>)